# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# create input file destination
input_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv(input_data_file, encoding="utf-8")
city_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lompoc,34.6391,-120.4579,62.78,89,90,4.61,US,1627475586
1,1,bereda,43.2686,-7.5406,71.42,62,6,5.06,ES,1627475819
2,2,nome,64.5011,-165.4064,51.87,93,90,23.02,US,1627475820
3,3,dikson,73.5069,80.5464,37.62,96,100,13.67,RU,1627475587
4,4,castro,-24.7911,-50.0119,40.93,56,53,8.08,BR,1627475821


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# set weight and locations
maxhumidity = humidity.max()
coordinates = city_df[["Lat", "Lng"]]

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(   coordinates, 
                                    weights=humidity, 
                                    dissipating=False, 
                                    max_intensity=maxhumidity,
                                    point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

city_df_new = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70) & (city_df["Wind Speed"]<10) & (city_df["Cloudiness"] == 0), :]
city_df_new = city_df_new.dropna(how='any')
city_df_new.reset_index(inplace=True)
del city_df_new['Unnamed: 0']
city_df_new

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,29,richards bay,-28.7830,32.0377,78.31,26,0,5.01,ZA,1627475842
1,61,tamiahua,21.2667,-97.4500,77.14,83,0,4.61,MX,1627475868
2,197,carlisle,40.2015,-77.1889,78.22,75,0,1.01,US,1627475995
3,247,porto nacional,-10.7081,-48.4172,77.00,24,0,4.97,BR,1627476041
4,327,chokwe,-24.5333,32.9833,78.73,30,0,6.55,MZ,1627476109
5,328,vila velha,-20.3297,-40.2925,77.97,67,0,9.22,BR,1627475814
6,413,zvishavane,-20.3267,30.0665,70.59,30,0,4.83,ZW,1627476186


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels = []

for i in range(len(city_df_new)):
    lat = city_df_new.loc[i]['Lat']
    lng = city_df_new.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params).json()

    try:
        hotels.append(requested['results'][0]['name'])
    except:
        hotels.append("")

# add a notel name to df
city_df_new["Hotel Name"] = hotels

# save info to hotels_df
hotels_df = city_df_new[["City","Country","Lat","Lng","Hotel Name"]]
hotels_df = hotels_df.dropna(how='any')
hotels_df.head()

,City,Country,Lat,Lng,Hotel Name
0,richards bay,ZA,-28.7830,32.0377,Richards Bay
1,tamiahua,MX,21.2667,-97.4500,Tamiahua
2,carlisle,US,40.2015,-77.1889,Carlisle
3,porto nacional,BR,-10.7081,-48.4172,Porto Nacional
4,chokwe,MZ,-24.5333,32.9833,Chokwe


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations2 = city_df_new[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations = locations2, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…